# Automatické derivování (autograd)

- Úkolem cvičení je naprogramovat algoritmus zpětné propagace pro obecný případ obdobným způsobem, jako je navržen v knihovně PyTorch.
- Implementujeme třídu `ans.autograd.Variable`, která bude imitovat návrh PyTorche až do verze [0.4.0](https://pytorch.org/blog/pytorch-0_4_0-migration-guide/), kdy došlo ke sloučení tříd `torch.autograd.Variable` a `torch.Tensor`.

**Poznámka**
- Tento notebook nezapíná [autoreload extension](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html), protože testování využívá type checking pomocí `isinstance` a [to při reloadu modulu selhává](https://github.com/ipython/ipython/issues/12399).
- Při každé modifikaci modulu `ans.autograd` je proto bohužel nutné notebook restartovat 😟

In [1]:
import sys
sys.path.append('..')  # import tests

import torch

import ans
from tests import test_autograd

# Třída `Variable`

- Třída `Variable` se bude chovat podobně jako `torch.Tensor` či `numpy.ndarray`, bude ovšem navíc obalená informacemi o výpočetním grafu.
- Třída `Variable` obsahuje atributy
  
  | atribut   | typ                                                  | role                                                            |
  |-----------|------------------------------------------------------|-----------------------------------------------------------------|
  | `data`    | `torch.Tensor`                                       | drží hodnotu proměnné                                           |
  | `grad`    | `torch.Tensor`                                       | obsahuje gradient na proměnnou                                  |
  | `parents` | `tuple[Variable, ...]`                               | odkazuje na přímé předky proměnné                               |
  | `grad_fn` | `Callable[[torch.Tensor], tuple[torch.Tensor, ...]]` | odkazuje na zpětný průchod operace, jejíž výsledkem proměnná je |

- Kostra třídy `Variable` je již připravená a lze vytvořit její instance.
- Vytvořme dvě proměnné `f` a `g`.

In [ ]:
f = ans.autograd.Variable(torch.tensor(2.))
f

In [ ]:
g = ans.autograd.Variable(torch.tensor(3.))
g

# Operace sčítání nad objekty typu `Variable`

- Nyní bychom chtěli, abychom mohli proměnné `f` a `g` např. sčítat jako
  ``` python
  h = f + g
  ```
- Výsledná proměnná `h` by měla být také typu `Variable` a její atributy by měly být
  
  | atribut   | hodnota                       | komentář                                                                                   |
  |-----------|-------------------------------|--------------------------------------------------------------------------------------------|
  | `data`    | `torch.tensor(5.)`            | 2 + 3 = 5                                                                                  |
  | `grad`    | `None`                        | ještě neproběhla zpětná propagace                                                          |
  | `parents` | `(f, g)`                      | `h` je potomkem `f` a `g`                                                                  |
  | `grad_fn` | `<function add_backward ...>` | funkce, která provede zpětný průchod operace sčítání v případě, že zavoláme `h.backward()` |

- V Pythonu lze násobení pro uživatelsky definovanou třídu definovat pomocí magické metody `__add__`, např.
  ``` python
  def __add__(self, other: Self) -> Self:
      def grad_fn(dout: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
          return dout.clone(), dout.clone()
      return self.__class__(
          self.data + other.data,
          parents = (self, other),
          grad_fn = grad_fn
      )
  ```

### TODO: implementujte metodu [`ans.autograd.Variable.__add__`](../ans/autograd.py) (prozatím pouze skaláry)

(stačí pouze zkopírovat kód)

In [ ]:
h = f + g
h

In [ ]:
print(h.data)  # tensor(5.)
print(h.parents[0] is f)  # True
print(h.parents[1] is g)  # True
print(h.grad_fn)  # <function ...> s jednim parametrem (prichozim gradientem)

- `grad_fn` je funkce zpětného průchodu sčítání, kterou můžeme zavolat s příchozím gradientem.
- Měli bychom dostat gradienty na vstupy (předky `h`).

In [ ]:
df, dg = h.grad_fn(torch.tensor(1.))  # prichozi gradient dh/dh = 1.0
print(df)  # tensor(1.) ... df znamena dh/df
print(dg)  # tensor(1.) ... dg znamena dh/dg

In [ ]:
test_autograd.TestAddScalars.eval()

# Odečítání `Variable`

- Obdobně můžeme definovat i operaci odečítání.
- Rozdíl bude pouze ve výpočtu `data` a gradient na druhý operand by měl být přenásoben minus jedničkou (`-dout`).

### TODO: implementujte metodu [`ans.autograd.Variable.__sub__`](../ans/autograd.py) (prozatím pouze skaláry)

In [ ]:
test_autograd.TestSubScalars.eval()

# Násobení `Variable`

- Ješte jednou pro násobení `h = f * g`.

### TODO: implementujte metodu [`ans.autograd.Variable.__mul__`](../ans/autograd.py) (prozatím pouze skaláry)

In [ ]:
test_autograd.TestMulScalars.eval()

# Dělení `Variable`

- A také  pro dělení `h = f / g`.

### TODO: implementujte metodu [`ans.autograd.Variable.__truediv__`](../ans/autograd.py) (prozatím pouze skaláry)

In [ ]:
test_autograd.TestDivScalars.eval()

# Operace nad tensory a built-in typy a podpora broadcastingu

Takto navržen bude kód fungovat jen v ideálních případech. Existují následující případy, kdy pravděpodobně selže.

1. **`Variable` * `float`**
- První případ, kdy se kód rozbije, je pokud argument `other` není `Variable`.
- Jedná se přitom o zcela očekávatelný use case.
- Příklad:
  ``` python
  v = ans.autograd.Variable(torch.tensor(2.))
  v * 3  # `AttributeError: 'int' object has no attribute 'data'`
  ```

2. **`float` * `Variable`**
- Násobení, kde první z operandů není `Variable`, pokrývá operátor `__rmul__`, nikoliv `__mul__`.
- Je proto nutné doplnit implementaci operátorů s `Variable` *na pravé straně*, tj. u `__rmul__`, `__radd__`, `__rsub__` a `__rtruediv__`.
- Příklad:
  ``` python
  v = ans.autograd.Variable(torch.tensor(2.))
  3 * v  #  NotImplementedError / TypeError: unsupported operand type(s) for *: 'float' and 'Variable'
  ```

3. **Broadcasting**
- Druhý případ může být, že `self.data` a `other.data` nebudou mít stejný rozměr, přitom ale budou ["broadcastable"](https://pytorch.org/docs/stable/notes/broadcasting.html).
- Jde tedy např. o vektor + skalár, matice + skalár, matice + vektor apod.
- Pokud dopředný průchod provádí broadcasting, je nutné ve zpětném průchodu gradient přes nadbytečné dimenze *sečíst*, aby výsledný gradient měl rozměr shodný s odpovídající proměnnou.
- Příklad:
  ``` python
  u = ans.autograd.Variable(torch.tensor([2., 3.]))  # (1, 2)
  v = ans.autograd.Variable(torch.tensor(3.))  # ()
  w = u * v  # (1, 2)
  du, dv = w.grad_fn()  # pokud dv = tensor([2., 3.]), neni to spravny vysledek, spravne je dv = tensor(5.0)
  ```

4. **`dtype` a `device`**
- Spíše okrajový případ je použití `dtype` jiného než `float32` a `device` jiné než `'cpu'`.
- Implementace by měla tyto parametry respektovat a neměla by přetypovávat ani přesouvat tensory na jiná `device`.
- Gradient by měl být rovněž vždy stejného typu a tvaru jako odpovídající `Variable`.

**Úkolem je nyní upravit a doplnit implementaci `Variable` tak, aby podporovala všechny popsané případy**.

*Poznámky:*
- Je možné (a doporučené) libovolně přidávat metody a funkce do třídy `Variable` či do souboru `autograd`, např. pro často opakující se kusy kódu.
- Takové metody považujte za privátní a jejich název začínejte jedním podtržítkem (např. `_my_aux_func`), abychom zamezili příp. kolizím jmen pro další cvičení.

### TODO: implementujte metody [`ans.autograd.Variable.__add__`](../ans/autograd.py) a [`ans.autograd.Variable.__radd__`](../ans/autograd.py)

In [ ]:
test_autograd.TestAddTensors.eval()

### TODO: implementujte metody [`ans.autograd.Variable.__sub__`](../ans/autograd.py) a [`ans.autograd.Variable.__rsub__`](../ans/autograd.py)

In [ ]:
test_autograd.TestSubTensors.eval()

### TODO: implementujte metody [`ans.autograd.Variable.__mul__`](../ans/autograd.py) a [`ans.autograd.Variable.__rmul__`](../ans/autograd.py)

In [ ]:
test_autograd.TestMulTensors.eval()

### TODO: implementujte metody [`ans.autograd.Variable.__truediv__`](../ans/autograd.py) a [`ans.autograd.Variable.__rtruediv__`](../ans/autograd.py)

In [ ]:
test_autograd.TestDivTensors.eval()

# Zpětná propagace

- Zpětnou propagaci, tj. výpočet gradientů na všechny vstupy libovolně složité funkce sestávající z operací výše, bude implementovat metoda `Variable.backprop`.
- Bude mít tedy obdobnou funkci jako metoda `torch.Tensor.backward` v PyTorchi, tzn. vyplní atribut `grad` správně vypočteným gradientem pro všechny proměnné předcházející v grafu proměnnou, ze které jsme `backprop` zavolali.
- Princip je vysvětlen v notebooku [pytorch_autograd](../notebooks/pytorch_autograd.ipynb).
- Jeden z možných postupů implementace je v sekci *"Reverzní automatické derivování"* v přednášce [ans-03-backprop](../slides/ans-03-backprop.pdf) (slide 57).
- Pro splnění testů je důležité, aby
  - seděly vypočtené gradienty a to *pro všechny proměnné v grafu*, nejen ty na začátku;
  - při zpětném průchodu grafem byla funkce pro výpočet lokálního řetízkového pravidla `grad_fn` pro každý uzel zavolána max. jednou. Nestačí tedy depth-first procházení grafu.

### TODO: implementujte metodu [`ans.autograd.Variable.backprop`](../ans/autograd.py)

In [ ]:
test_autograd.TestBackprop.eval()